In [ ]:
import os
os.chdir('../daizika')

In [ ]:
from edgar import SECHelper, SECEdgarParser

In [30]:
import pandas as pd
from datetime import datetime
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline  

# Code to generate stats from CIK Lookup file

In [ ]:
cikObj = SECHelper.CIKDownloader()
cikObj.download_cik_file()
cik_folder, cik_file = cikObj.get_cik_target_filename()

In [ ]:
companies = {}
with open(cik_file, mode='r', encoding='utf-8') as f:
    for line in f:
        cik_parts = line.rstrip('\n').split(':', 3)
        companies[cik_parts[1]] = cik_parts[0]
print(len(companies))

# Code to generate stats from form files

In [39]:
# Download Quarterly Form Index file
start_index_year = 2000
qtrs = [1, 2, 3, 4]
#forms=['4', '8-K', '10-K', '10-Q']
form = '10-K'
formDownloader = SECHelper.FormDownloader()
data = []
labels = ['dt_file', 'year', 'month', 'day', 'form_id', 'company_name', 'cik']
for year in range(0, 18):
    index_year = start_index_year + year
    for index_qtr in qtrs:
        target_folder, target_file = formDownloader.get_qtr_form_idx_target_file(index_year, index_qtr)
        print('Processing {}/{} {}'.format(index_year, index_qtr, target_file))
        with open(target_file, mode="r", encoding="utf-8") as idx:
            bStart = False                    
            for line in idx:
                if not bStart and line.startswith('-----'):
                    bStart = True
                elif bStart:
                    fields = [token.strip() for token in line.rstrip('\n').split('  ') if len(token.strip()) > 0]
                    if fields[0] == form:
                        #print(line)
                        form_id = line[0:12].strip()
                        company_name = line[12:74].strip()
                        cik = line[74:86].strip()
                        filing_dt = line[86:98].strip().replace('-', '')
                        year = int(filing_dt[0:4])
                        month = int(filing_dt[4:6])
                        day = int(filing_dt[6:8])
                        dt_file = datetime(year, month, day)
                        data.append([dt_file, year, month, day, form_id, company_name, cik])
        print('...done')
df = pd.DataFrame.from_records(data, columns=labels)        

Processing 2000/1 /development/data/sec/edgar/index/full/form_2000_QTR1.idx
...done
Processing 2000/2 /development/data/sec/edgar/index/full/form_2000_QTR2.idx
...done
Processing 2000/3 /development/data/sec/edgar/index/full/form_2000_QTR3.idx
...done
Processing 2000/4 /development/data/sec/edgar/index/full/form_2000_QTR4.idx
...done
Processing 2001/1 /development/data/sec/edgar/index/full/form_2001_QTR1.idx
...done
Processing 2001/2 /development/data/sec/edgar/index/full/form_2001_QTR2.idx
...done
Processing 2001/3 /development/data/sec/edgar/index/full/form_2001_QTR3.idx
...done
Processing 2001/4 /development/data/sec/edgar/index/full/form_2001_QTR4.idx
...done
Processing 2002/1 /development/data/sec/edgar/index/full/form_2002_QTR1.idx
...done
Processing 2002/2 /development/data/sec/edgar/index/full/form_2002_QTR2.idx
...done
Processing 2002/3 /development/data/sec/edgar/index/full/form_2002_QTR3.idx
...done
Processing 2002/4 /development/data/sec/edgar/index/full/form_2002_QTR4.idx


In [36]:
df.columns

Index(['dt_file', 'year', 'month', 'day', 'form_id', 'company_name', 'cik'], dtype='object')

In [42]:
#df[df.year == 2017].groupby(['year', 'month']).cik.nunique()
df[df.year == 2017].cik.nunique()

7074